## Run SimSS simulations and plot the results
Version 0.0.1
(c) Vincent M. Le Corre (https://github.com/VMLC-PV)

In [ ]:
# Package import
# %matplotlib inline
import os,platform,warnings,itertools,matplotlib,copy
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
# Import homemade package by VLC
from SIMsalabim_utils.RunSim import *
from SIMsalabim_utils.SimSS_plots import *
from SIMsalabim_utils.aux_func import *
from SIMsalabim_utils.plot_settings_screen import *

## General Inputs
warnings.filterwarnings("ignore")           # Don't show warnings
system = platform.system()                  # Operating system
max_jobs = 5                 # Max number of parallel simulations (for number of CPU use: os.cpu_count() )
do_multiprocessing = False                      # Use multiprocessing
if system == 'Windows':                     # cannot easily do multiprocessing in Windows
        max_jobs = 1
        do_multiprocessing = False
        try:                                # kill all running jobs to avoid conflicts
            os.system('taskkill.exe /F /IM simss.exe')
        except:
            pass
matplotlib.rcParams['figure.figsize'] = (7, 5) # set default figure size
# Path to SIMsalabim
path2simu = os.path.join(os.getcwd() , 'Simulation_program','SIMsalabimv456','SimSS')

In [ ]:
def DoSim(parameters,fixed_str = '',run_simu = True,SimuInputsFunc = PrepareSimuInputs,param2plot_name='',param2plot_values=[],plot_type=1,plot_ideality=False,leg_labels=[],figname=''):
    """Run simulations and plot results

    Parameters
    ----------
    param_name : str
        str or list of str with parameter names to be varied
    param_values : 1-D sequence of floats
        1-D sequence of values or list of list with the values for the parameters to be varied
    Gfracs : 1-D sequence of floats
        list with the light-intensity fractions to be varied
    fixed_str : str, optional
        string with the fixed parameters to be passed to the simulation, by default ''
    run_simu : bool, optional
        whether to run the simulations, by default True
    SimuInputsFunc : func, optional
        function to prepare the inputs for the simulations, by default PrepareSimuInputs
    param2plot_name : str, optional
        str with the name of the parameter to be plotted, by default ''
    param2plot_values : list, optional
        list with the values of the parameter to be plotted, by default []
    plot_type : int, optional
        type of plot 1 = logx, 2 = logy, 3 = loglog else linlin (default = linlin), by default 0
    plot_ideality : bool, optional
        whether to plot the ideality factor, by default False
    leg_labels : list, optional
        list with the labels for the legend, by default []
    figname : str, optional
        begining of the name of the figure to be saved, by default ''
    """
    # Specifiy the folder to save the simulation images   
    scpars = []
    Vocs,Jscs,FFs,PCEs = [],[],[],[]

    Simulation_Inputs = SimuInputsFunc(path2simu,parameters=parameters,fixed_str=fixed_str,CodeName = 'SimSS',SaveVarFile=False,verbose=False) # creates all necessary list inputs
    
    if run_simu:
        RunSimulation(Simulation_Inputs,max_jobs=max_jobs,do_multiprocessing=do_multiprocessing,verbose=False) # runs the simulations

    # get Gfracs from parameters
    Gfracs = []
    for i in parameters:
        if i['name'] == 'Gfrac':
            Gfracs = i['values']
    
    parameters_copy = copy.deepcopy(parameters)
    for G in Gfracs:
        for i in parameters_copy:
            if i['name'] == 'Gfrac':
                i['values'] = [G]
        Simulation_Inputs = SimuInputsFunc(path2simu,parameters=parameters_copy,fixed_str=fixed_str,CodeName = 'SimSS',verbose=False)

        str_lst,JV_files,Var_files,scPars_files,code_name_lst,path_lst,labels = Simulation_Inputs

        if G == 1:
            JV2plot = JV_files
            scPars2plot = scPars_files
        scpars.append(scPars_files)

        Voc_dum,Jsc_dum,FF_dum,PCE_dum = [],[],[],[]
        for i in scPars_files:
            df = pd.read_csv(i,delim_whitespace=True)
            Voc_dum.append(float(df['Voc']))
            Jsc_dum.append(float(abs(df['Jsc'])))
            FF_dum.append(float(df['FF']))
            PCE_dum.append(float(abs(df['MPP']/G)))
        Vocs.append(Voc_dum)
        Jscs.append(Jsc_dum)
        FFs.append(FF_dum)
        PCEs.append(PCE_dum)
    Jscs = np.asarray(Jscs)
    FFs = np.asarray(FFs)
    PCEs = np.asarray(PCEs)
    Vocs = np.asarray(Vocs)
    

    if param2plot_name=='' or param2plot_values==[]:
        print('No parameter to plot')
    else:
        param2plot = param2plot_values
        plt.figure(0,figsize=(14,10))
        colors = plt.cm.viridis(np.linspace(0,1,max(len(param2plot),3)+1)) # prepare color for plots
        nID = []
        for idx in range(len(param2plot)):
            plt.subplot(221)
            plt.semilogx(Gfracs,Vocs[:,idx],'-',color=colors[idx])
            if plot_ideality:
                res = get_ideality_factor(np.asarray(Gfracs),list(Vocs[:,idx]),T=295)
                nID.append(res[0])
            plt.subplot(222) 
            plt.loglog(Gfracs, Jscs[:,idx]/10,'-',color=colors[idx]) # set in mA/cm^2
            plt.subplot(223)
            plt.semilogx(Gfracs,FFs[:,idx],'-',color=colors[idx])
            plt.subplot(224)
            plt.semilogx(Gfracs,PCEs[:,idx]/10,'-',color=colors[idx]) # set in %
        plt.subplot(221)
        plt.xlabel('Light Intensity [suns]')
        plt.ylabel('V$_{OC}$ [V]')
        plt.subplot(222)
        plt.xlabel('Light Intensity [suns]')
        plt.ylabel('J$_{SC}$ [mA cm$^2$]')
        plt.subplot(223)
        plt.xlabel('Light Intensity [suns]')
        plt.ylabel('FF')
        plt.subplot(224)
        # plt.legend(title=param2plot_name,loc='best')
        plt.xlabel('Light Intensity [suns]')
        plt.ylabel('PCE [%]')
        plt.tight_layout()
        #plot 1 sun only
        colors = plt.cm.viridis(np.linspace(0,1,max(len(str_lst),3)+1)) # prepare color for plots
        Gfrac = np.ones(len(param2plot)) # keep at 1 sun here
        PlotJV(JV2plot, labels=leg_labels,colors=colors,num_fig=1,plot_type=0,x='Vext',y=['Jext'],legend=True,xlimits=[0,1.5],ylimits=[-26,1e1],x_unit='V',y_unit='mA/cm^2',save_fig=True,fig_name='JV.pdf',mark='',line_type = ['-'])
        plt.legend(title=param2plot_name,loc='best',ncol=1)
        plt.tight_layout()
        PlotJVPerf(param2plot, scPars_files,y='Jsc',Gfrac=Gfrac,plot_type=plot_type,xlimits=[],ylimits=[0.,1.1],xlabel=param2plot_name,ylabel='Norm. J$_{SC}$',legend='',num_fig=2,mark='o',line_type = ['-'],norm_plot=True,save_fig=True,fig_name=figname+'Jsc.svg')
        plt.tight_layout()
        PlotJVPerf(param2plot, scPars_files,y='Voc',Gfrac=Gfrac,plot_type=plot_type,xlimits=[],ylimits=[0.,1.1],xlabel=param2plot_name,ylabel='Norm. V$_{OC}$',legend='',num_fig=3,mark='o',line_type = ['-'],norm_plot=True,save_fig=True,fig_name=figname+'Voc.svg')
        plt.tight_layout()
        PlotJVPerf(param2plot, scPars_files,y='FF',Gfrac=Gfrac,plot_type=plot_type,xlimits=[],ylimits=[0.,1.1],xlabel=param2plot_name,ylabel='Norm. FF',legend='',num_fig=4,mark='o',line_type = ['-'],norm_plot=True,save_fig=True,fig_name=figname+'FF.svg')
        plt.tight_layout()
        PlotJVPerf(param2plot, scPars_files,y='PCE',Gfrac=Gfrac,plot_type=plot_type,xlimits=[],ylimits=[0.,1.1],xlabel=param2plot_name,ylabel='Norm. PCE',legend='',num_fig=5,mark='o',line_type = ['-'],norm_plot=True,save_fig=True,fig_name=figname+'PCE.svg')
        plt.tight_layout()
        if plot_ideality:
            plt.figure(7)  
            if plot_type == 1:
                plt.semilogx(param2plot,nID,'-o')
            elif plot_type == 2:
                plt.semilogy(param2plot,nID,'-o')
            elif plot_type == 3:
                plt.loglog(param2plot,nID,'-o')
            else:
                plt.plot(param2plot,nID,'-o')
            
            plt.xlabel(param2plot_name)
            plt.ylabel('$\eta$')
            plt.ylim([0,3])
            plt.tight_layout()
            plt.show()

In [ ]:
# Low absoprtion i.e. low Gehp
abs_low = True
if abs_low:
    num_sim = 5
    param_name = 'Gehp'
    fixed_str = ''
    param_values = list(np.linspace(2.5e27,3.5e27,num_sim))
    parameters = [{'name':param_name,'values':list(param_values)}]
    leg_labels = [sci_notation(x, sig_fig=2) for x in param_values]
    Gfracs = list(np.geomspace(1e-2,3,10))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='Gehp [m$^{-3}$ s$^{-1}$]',param2plot_values = param_values,plot_type=0,leg_labels=leg_labels,figname='Gehp',plot_ideality=True)

In [ ]:
# Low active layer thickness
thick_low = True
if thick_low:
    num_sim = 5
    param_name = 'L'
    fixed_str = ''
    param_values = list(np.linspace(340e-9,540e-9,num_sim))
    parameters = [{'name':param_name,'values':list(param_values)}]
    Lac = (np.linspace(340e-9,540e-9,num_sim) - 40e-9)*1e9 # remove the thickness for the TL as L is the total thickness
    leg_labels = ['{:.0f}'.format(x) for x in Lac]
    Gfracs = list(np.geomspace(1e-2,3,10))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='Active Layer Thickness [nm]',param2plot_values=Lac,plot_type=0,leg_labels=leg_labels,figname='AlT',plot_ideality=True)

In [ ]:
# Low shunt resistance
shunt = True
if shunt:
    num_sim = 6
    param_name = 'Rshunt'
    fixed_str = '-Vstep 1e-3'
    param_values = list(np.geomspace(1e-2,1e3,num_sim))
    parameters = [{'name':param_name,'values':list(param_values)}]
    leg_labels = [sci_notation(x, sig_fig=0) for x in param_values]
    Gfracs = list(np.geomspace(1e-3,3,10))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='Shunt Resistance [$\Omega$ m$^2$]',param2plot_values = param_values,plot_type=1,leg_labels=leg_labels,figname='Rshunt',plot_ideality=True)
    

    # plot dark JV
    fixed_str = '-Vstep 1e-3 -Gfrac 0 -Vmax 1.4'
    parameters = []
    parameters.append({'name':param_name,'values':param_values}) 
    Simulation_Inputs = PrepareSimuInputs(path2simu,parameters=parameters,fixed_str='device_parameters_pero.txt '+fixed_str,CodeName = 'SimSS',verbose=True) # creates all necessary list inputs
    run_simu = True
    if run_simu:
        RunSimulation(Simulation_Inputs,max_jobs=max_jobs,do_multiprocessing=do_multiprocessing,verbose=False) # runs the simulations

    colors = plt.cm.viridis(np.linspace(0,1,max(len(param_values),3)+1)) # prepare color for plots
    
    PlotJV(Simulation_Inputs[1], leg_labels,colors=colors,num_fig=6,plot_type=2,x='Vext',y=['Jext'],legend=False,xlimits=[0,1.4],ylimits=[1e-6,1e4],x_unit='V',y_unit='mA/cm^2',save_fig=True,fig_name=os.path.join(path2simu,'JV.svg'),mark='',line_type = ['-'])
        

In [ ]:
# Pinned carrier density - TL doping
pcd_tldop = True
if pcd_tldop:
    num_sim = 5
    param_name = 'doping_RTL'
    fixed_str = '-Rshunt -1 -L_RTL 20e-9 -Bulk_tr 1e0 -St_L 1e0 -mob_RTL 1.5e-8 -Vmax 1.5 -St_R 1e17'
    param_values = list(np.geomspace(1e21,1e25,5))
    parameters = [{'name':param_name,'values':list(param_values)}]
    leg_labels = [sci_notation(x, sig_fig=0) for x in param_values]
    Gfracs = list(np.geomspace(1e-2,5,10))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='TL Doping Density [m$^{-3}$]',param2plot_values = param_values,plot_type=1,leg_labels=leg_labels,figname='pcd_dop_tl',plot_ideality=True)

In [ ]:
# Pinned carrier density - L_R_RTL
pcd = True
if pcd:
    num_sim = 5
    param_name = 'L_RTL'
    fixed_str = '-St_R 1e17 -St_L 1e0 -doping_RTL 1e0 -Rshunt -1 -Bulk_tr 1e0'
    param_values = ([1e-9, 2e-9, 5e-9, 10e-9, 20e-9])
    parameters = [{'name':param_name,'values':list(param_values)}]
    Lac =[i*1e9 for i in param_values] 
    leg_labels = ['{:.0f}'.format(x) for x in Lac]
    Gfracs = list(np.geomspace(1e-2,5,10))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='TL Thickness [nm]',param2plot_values = Lac,plot_type=0,leg_labels=leg_labels,figname='pcd_l_TL',plot_ideality=True)

In [ ]:
# band-to-band/Bimolecular recombination
rec_b2b = False
if rec_b2b:
    num_sim = 6
    param_name = 'kdirect'
    fixed_str = '-Bulk_tr 0 -St_L 0 -St_R 0 -Rseries 0 -Rshunt -1e-5 -Vmax 1.6' # uncomment this line to remove effect of resistance
    param_values = list(np.geomspace(1e-15,1e-10,num_sim))
    parameters = [{'name':param_name,'values':list(param_values)}]
    leg_labels = [sci_notation(x, sig_fig=0) for x in param_values]
    Gfracs = list(np.geomspace(1e-2,3,10))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='k$_{direct}$ [m$^3$ s$^{-1}]$',param2plot_values = param_values,plot_type=1,leg_labels=leg_labels,figname='kdirect',plot_ideality=True)


In [ ]:
# Traps
Traps = True
if Traps:
    num_sim = 5
    #fixed_str = '-St_L 0 -St_R 0 -Vmax 1.6 -Vstep 1e-2' # Pero line
    fixed_str = '-Rshunt -1e0 -Rseries 0 ' # uncomment this line to remove effect of resistance
    # fixed_str = ''
    param_name = 'Bulk_tr'
    param_values = list(np.geomspace(1e19,1e23,num_sim))
    parameters = [{'name':param_name,'values':list(param_values)}]
    leg_labels = [sci_notation(x, sig_fig=0) for x in param_values]
    Gfracs = list(np.geomspace(1e-2,3,10))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='Bulk Trap Density [m$^{-3}$]',param2plot_values = param_values,plot_type=1,leg_labels=leg_labels,figname='Bulk_tr',plot_ideality=True)

In [ ]:
# Trap depth
Trap_depth = True
if Trap_depth:
    num_sim = 6
    fixed_str = '-Bulk_tr 5e22 -Rshunt -1e0 -Rseries 0 -kdirect 1e-20 -Vmax 1.4' # uncomment this line to remove effect of resistance
    #fixed_str = '-Bulk_tr 5e21 '
    param_name = 'EtrapSingle'
    param_values = list(np.linspace(4.0,4.5,num_sim))
    parameters = [{'name':param_name,'values':list(param_values)}]
    offset = list(np.linspace(4.0,4.5,num_sim) - 3.9)
    leg_labels = ['{:.1f}'.format(x) for x in offset]
    Gfracs = list(np.geomspace(1e-2,5,10))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='Trap Depth [eV]',param2plot_values = offset,plot_type=0,leg_labels=leg_labels,figname='Trap_depth',plot_ideality=True)

In [ ]:
# Interface Traps
Int_Traps = True
if Int_Traps:
    num_sim = 5
    # fixed_str = ''
    fixed_str = '-Rshunt -1e0 -Rseries 0' # uncomment this line to remove effect of resistance
    St = list(np.geomspace(1e13,1e17,num_sim))
    param_name = ['St_L','St_R']
    param_values = [St,St]
    parameters = [{'name':param_name[0],'values':list(param_values[0])},{'name':param_name[1],'values':list(param_values[1])}]
    leg_labels = [sci_notation(x, sig_fig=0) for x in St]
    Gfracs = list(np.geomspace(1e-2,3,6))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='Interface Traps [m$^2$]',param2plot_values = St,plot_type=1,leg_labels=leg_labels,figname='Int_tr',plot_ideality=True)

In [ ]:
# energy level alignment no interface traps
align_el = True
if align_el:
    num_sim = 5
    fixed_str = '-St_L 0 -St_R 0 -VB_LTL 6.2 -Bulk_tr 0 -Vmax 1.5 -mun_0 1e-5'
    W_L = list(np.linspace(3.9,4.3,num_sim))
    CB_LTL = W_L 
    offset = list(np.linspace(3.9,4.3,num_sim) - 3.9)
    leg_labels = ['{:.1f}'.format(x) for x in offset]
    param_name = ['W_L','CB_LTL']
    param_values = [W_L,CB_LTL]
    parameters = [{'name':param_name[0],'values':list(param_values[0])},{'name':param_name[1],'values':list(param_values[1])}]
    Gfracs = list(np.geomspace(1e-2,3,6))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='Injection barrier [eV]',param2plot_values = offset,plot_type=0,leg_labels=leg_labels,figname='align_el_low_mob',plot_ideality=True)

In [ ]:
# energy level alignment with interface traps
align_el = True
if align_el:
    num_sim = 5
    fixed_str = ''
    W_L = list(np.linspace(3.9,4.3,num_sim))
    CB_LTL = W_L
    offset = list(np.linspace(3.9,4.3,num_sim) - 3.9)
    leg_labels = ['{:.1f}'.format(x) for x in offset]
    param_name = ['W_L','CB_LTL']
    param_values = [W_L,CB_LTL]
    parameters = [{'name':param_name[0],'values':list(param_values[0])},{'name':param_name[1],'values':list(param_values[1])}]
    Gfracs = list(np.geomspace(1e-2,3,6))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='Injection barrier [eV]',param2plot_values = offset,plot_type=0,leg_labels=leg_labels,figname='align_el_int_trap',plot_ideality=True)

In [ ]:
# energy level alignment no interface traps
align_el = True
if align_el:
    num_sim = 5
    fixed_str = '-St_L 0 -St_R 0 -VB_LTL 6.2 -Bulk_tr 0 -Vmax 1.5' #
    W_L = list(np.linspace(3.9,4.3,num_sim))
    CB_LTL = W_L 
    offset = list(np.linspace(3.9,4.3,num_sim) - 3.9)
    leg_labels = ['{:.1f}'.format(x) for x in offset]
    param_name = ['W_L','CB_LTL']
    param_values = [W_L,CB_LTL]
    parameters = [{'name':param_name[0],'values':list(param_values[0])},{'name':param_name[1],'values':list(param_values[1])}]
    Gfracs = list(np.geomspace(1e-2,3,6))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='Injection barrier [eV]',param2plot_values = offset,plot_type=0,leg_labels=leg_labels,figname='align_el',plot_ideality=True)

In [ ]:
# Concentrtion of ions
Ions_con_np = True
if Ions_con_np:
    num_sim = 5
    fixed_str = '-mob_ion_spec 0'
    param_name = ['CNI','CPI']
    param_values = [list(np.geomspace(1e19,1e23,num_sim)),list(np.geomspace(1e19,1e23,num_sim))]
    parameters = [{'name':param_name[0],'values':list(param_values[0])},{'name':param_name[1],'values':list(param_values[1])}]
    leg_labels = [sci_notation(x, sig_fig=0) for x in list(np.geomspace(1e19,1e23,num_sim))]
    Gfracs = list(np.geomspace(1e-2,3,6))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='Ion Conc. [m$^{-3}$]',param2plot_values = list(np.geomspace(1e19,1e23,num_sim)),plot_type=1,leg_labels=leg_labels,figname='CI',plot_ideality=True)

In [ ]:
# Concentrtion of ions , ions can move into TL IonsInTLs 1'
Ions_con_np = True
if Ions_con_np:
    num_sim = 5
    fixed_str = '-mob_ion_spec 0 -IonsInTLs 1'
    param_name = ['CNI','CPI']
    param_values = [list(np.geomspace(1e19,1e23,num_sim)),list(np.geomspace(1e19,1e23,num_sim))]
    parameters = [{'name':param_name[0],'values':list(param_values[0])},{'name':param_name[1],'values':list(param_values[1])}]
    leg_labels = [sci_notation(x, sig_fig=0) for x in list(np.geomspace(1e19,1e23,num_sim))]
    Gfracs = list(np.geomspace(1e-2,3,6))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='Ion Conc. [m$^{-3}$]',param2plot_values = list(np.geomspace(1e19,1e23,num_sim)),plot_type=1,leg_labels=leg_labels,figname='CI_TL_On',plot_ideality=True)

In [ ]:
# Ion Ratio
conc_neg_ions = True
if conc_neg_ions:
    num_sim = 5
    param_name = 'CNI'
    fixed_str = '-Vmax 1.8 -mob_ion_spec 0'
    param_values = list(np.geomspace(1e19,1e23,num_sim))
    parameters = [{'name':param_name,'values':list(param_values)}]
    ratios = [x / 1e22 for x in param_values]
    leg_labels = [sci_notation(x, sig_fig=0) for x in ratios]
    Gfracs = list(np.geomspace(1e-2,5,10))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='Ion Conc. Ratio',param2plot_values = param_values,plot_type=1,leg_labels=leg_labels,figname='CIR',plot_ideality=True)

In [ ]:
# Concentration negative ions, no pos ions
conc_neg_ions_pi0 = True
if conc_neg_ions_pi0 :
    num_sim = 5
    param_name = 'CNI'
    fixed_str = '-Vmax 1.8 -mob_ion_spec -1 -CPI 0'
    param_values = list(np.geomspace(1e19,1e23,num_sim))
    parameters = [{'name':param_name,'values':list(param_values)}]
    leg_labels = [sci_notation(x, sig_fig=0) for x in param_values]
    Gfracs = list(np.geomspace(1e-2,5,10))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='Conc. Neg. Ions [$m^{-3}$]',param2plot_values = param_values,plot_type=1,leg_labels=leg_labels,figname='CNI_P0',plot_ideality=True)

In [ ]:
# High series resistance
series = True
if series:
    num_sim = 5
    param_name = 'Rseries'
    fixed_str = '-Vstep 1e-3 -Rshunt 5e1'
    param_values = list(np.geomspace(1e-6,1e-2,num_sim))
    parameters = [{'name':param_name,'values':list(param_values)}]
    leg_labels = [sci_notation(x, sig_fig=0) for x in param_values]
    Gfracs = list(np.geomspace(1e-2,3,10))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='Series Resistance [$\Omega$ m$^2$]',param2plot_values = param_values,plot_type=1,leg_labels=leg_labels,figname='Rseries',plot_ideality=True)
    
    # plot dark JV
    fixed_str = '-Vstep 5e-2 -Gfrac 0 -Vmax 1.4 -Rshunt 5e1'
    parameters = []
    parameters.append({'name':param_name,'values':param_values}) 
    Simulation_Inputs = PrepareSimuInputs(path2simu,parameters=parameters,fixed_str= 'device_parameters_pero.txt '+fixed_str,CodeName = 'SimSS',verbose=True) # creates all necessary list inputs
    run_simu = True
    if run_simu:
        RunSimulation(Simulation_Inputs,max_jobs=max_jobs,do_multiprocessing=do_multiprocessing,verbose=False) # runs the simulations

    colors = plt.cm.viridis(np.linspace(0,1,max(len(param_values),3)+1)) # prepare color for plots
    
    PlotJV(Simulation_Inputs[1], leg_labels,colors=colors,num_fig=6,plot_type=2,x='Vext',y=['Jext'],legend=False,xlimits=[0,1.4],ylimits=[1e-5,1e4],x_unit='V',y_unit='mA/cm^2',save_fig=True,fig_name=os.path.join(path2simu,'JV.svg'),mark='',line_type = ['-'])

In [ ]:
# Mobility TL
TL_mob = True
if TL_mob:
    num_sim = 6
    fixed_str ='-Vmax 1.3'
    param_name = 'mob_LTL'
    param_values = list(np.geomspace(1e-9,1e-4,num_sim))
    parameters = [{'name':param_name,'values':list(param_values)}]
    leg_labels = [sci_notation(x, sig_fig=0) for x in param_values]
    Gfracs = list(np.geomspace(1e-2,5,10))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='TL Mobility [m$^2$ V$^{-1}$ s$^{-1}$]',param2plot_values = param_values,plot_type=1,leg_labels=leg_labels,figname='TL_mob',plot_ideality=True)

In [ ]:
# Doping TL
TL_dop = True
if TL_dop:
    num_sim = 5
    param_name = 'doping_LTL'
    fixed_str = '-mob_LTL 1e-8 -L_LTL 40e-9  -L 450e-9'
    param_values = list(-np.geomspace(1e20,1e24,num_sim))
    parameters = [{'name':param_name,'values':list(param_values)}]
    leg_labels = [sci_notation(x, sig_fig=0) for x in param_values]
    Gfracs = list(np.geomspace(1e-2,5,10))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='TL Doping Density [m$^{-3}$]',param2plot_values = param_values,plot_type=1,leg_labels=leg_labels,figname='TL_dop',plot_ideality=True)
    

In [ ]:
# Work function electrode
WF = True
if WF:
    num_sim = 6
    param_name = 'W_L'
    fixed_str = '-Vmax 1.3'
    param_values = list(np.linspace(3.9,4.4,num_sim))
    parameters = [{'name':param_name,'values':list(param_values)}]
    offset = list(np.linspace(3.9,4.4,num_sim) - 3.9)
    leg_labels = ['{:.1f}'.format(x) for x in offset]
    Gfracs = list(np.geomspace(1e-2,5,10))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='Elec. WF Offset [eV]',param2plot_values = offset,plot_type=0,leg_labels=leg_labels,figname='Wf',plot_ideality=True)

In [ ]:
# Tranfer velocities TL/active layer
nu_int = True
if nu_int:
    num_sim = 5
    param_name = 'nu_int_LTL'
    fixed_str = ''
    param_values = list(np.geomspace(1e-4,1e4,num_sim))
    parameters = [{'name':param_name,'values':list(param_values)}]
    leg_labels = [sci_notation(x, sig_fig=0) for x in param_values]
    Gfracs = list(np.geomspace(1e-2,5,10))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='TL/AL Transfer\n Velocity [m s$^{-1}$]',param2plot_values = param_values,plot_type=1,leg_labels=leg_labels,figname='nu_int',plot_ideality=True)

In [ ]:
# Extraction barrier
align_el = True
if align_el:
    num_sim = 5
    fixed_str = '-Vmax 1.3'
    CB_LTL = np.linspace(3.9,3.5,num_sim)
    offset = list(3.9 - np.linspace(3.9,3.5,num_sim))
    leg_labels = ['{:.1f}'.format(x) for x in offset]
    param_name = 'CB_LTL'
    param_values = CB_LTL
    parameters = [{'name':param_name,'values':list(param_values)}]
    Gfracs = list(np.geomspace(1e-2,3,6))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='Extraction Barrier [eV]',param2plot_values = offset,plot_type=0,leg_labels=leg_labels,figname='extr_bar',plot_ideality=True)

In [ ]:
# Low active layer mobility
low_mob = True
if low_mob:
    num_sim = 5
    fixed_str = '-Vmax 1.3'
    param_name = ['mun_0','mup_0']
    param_values = [list(np.geomspace(1e-5,1e-3,num_sim)),list(np.geomspace(1e-7,1e-3,num_sim))]
    parameters = [{'name':param_name[0],'values':list(param_values[0])},{'name':param_name[1],'values':list(param_values[1])}]
    leg_labels = [sci_notation(x, sig_fig=0) for x in list(np.geomspace(1e-7,1e-3,num_sim))]
    Gfracs = list(np.geomspace(1e-2,3,6))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='Mobility [m$^2$ V$^{-1}$ s$^{-1}$]',param2plot_values = list(np.geomspace(1e-10,1e-6,num_sim)),plot_type=1,leg_labels=leg_labels,figname='low_mob',plot_ideality=True)
    

In [ ]:
# Mobility ratio
ratio_mob = True
if ratio_mob:
    fixed_str =''
    #mun_0 = [1e-4,1e-4,1e-4,1e-4]
    #mup_0 = [1e-7,1e-6,1e-5,1e-4]
    mun_0 = [1e-3, 1e-3,1e-3,1e-3,1e-3]
    mup_0 = [1e-7,1e-6,1e-5,1e-4,1e-3]
    
    ratio = [1e4, 1e3,1e2,1e1,1]
    param_name = ['mun_0','mup_0']
    param_values = [mun_0,mup_0]
    parameters = [{'name':param_name[0],'values':list(param_values[0])},{'name':param_name[1],'values':list(param_values[1])}]
    leg_labels = [sci_notation(x, sig_fig=0) for x in ratio]
    Gfracs = list(np.geomspace(1e-2,3,6))
    if 1 not in Gfracs:
        Gfracs.append(1)
        Gfracs.sort()
    parameters.append({'name':'Gfrac','values':Gfracs})
    DoSim(parameters, fixed_str= 'device_parameters_pero.txt '+fixed_str,SimuInputsFunc=DegradationPrepareSimuInputs_light_dep,run_simu=True,param2plot_name='Mobility Ratio',param2plot_values = ratio,plot_type=1,leg_labels=leg_labels,figname='ratio_mob',plot_ideality=True)

In [ ]:
# Clean output files from simulation folders
from SIMsalabim_utils.CleanFolder import *

Do_Cleaning = True # Careful, this will delete all files in the folder
if Do_Cleaning:
    clean_up_output('tj',path2simu)
    clean_up_output('tVG',path2simu)
    clean_up_output('JV',path2simu)
    clean_up_output('Var',path2simu)
    clean_up_output('scPars',path2simu)
    clean_up_output('Str',path2simu)
    clean_up_output('log',path2simu)
    # clean_up_output('link',path2simu)